<a href="https://colab.research.google.com/github/Vedmani/wear-particle-classification/blob/main/CNN_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install wandb -qU
!wandb login

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
import wandb
from wandb.keras import WandbCallback
from tensorflow.keras import layers

In [ ]:
tf.keras.utils.set_random_seed(42)
tf.config.experimental.enable_op_determinism()

In [ ]:
PATH = '/content/drive/MyDrive/Research Internship/dataset/wear particles/Train'
test_path = '/content/drive/MyDrive/Research Internship/dataset/wear particles/Test'
batch_size = 32
img_height = 224
img_width = 224

In [ ]:
train_path = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="training",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="validation",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

test_ds = tf.keras.utils.image_dataset_from_directory(
  test_path,
  seed = 42,
  image_size=(img_height, img_width)  )

Found 868 files belonging to 5 classes.
Using 695 files for training.
Found 868 files belonging to 5 classes.
Using 173 files for validation.
Found 99 files belonging to 5 classes.


In [ ]:
class_names = train_ds.class_names
test_class_names = test_ds.class_names
print(class_names,test_class_names)

['cutting wear', 'normal rubbing and cutting wear combined', 'normal rubbing wear', 'red oxide', 'severe sliding wear'] ['cutting wear', 'normal rubbing and cutting wear combined', 'normal rubbing wear', 'red oxide', 'severe sliding wear']


In [ ]:
num_classes = len(class_names)
data_augmentation = tf.keras.Sequential([tf.keras.layers.RandomFlip("horizontal_and_vertical"),tf.keras.layers.RandomRotation(0.2),tf.keras.layers.RandomZoom(0.1),])
model = Sequential([
  tf.keras.layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  data_augmentation,
  layers.Conv2D(16, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(128, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(256, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(512, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(10, activation='relu'),
  layers.Dropout(0.5), 
  layers.Dense(num_classes, activation='softmax')
])

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_2 (Rescaling)     (None, 240, 240, 3)       0         
                                                                 
 sequential_4 (Sequential)   (None, 240, 240, 3)       0         
                                                                 
 conv2d_6 (Conv2D)           (None, 238, 238, 16)      448       
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 119, 119, 16)     0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 117, 117, 32)      4640      
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 58, 58, 32)       0         
 2D)                                                  

In [ ]:
'''model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="/content/drive/MyDrive/Research Internship/augmentation/my_best_model.epoch{epoch:02d}-val_accuracy{val_accuracy:.2f}",
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)'''

In [ ]:
wandb.init(
      # Set the project where this run will be logged
      project="CNN_particle_classification", 
      name = "run-1",
)

In [ ]:
%%time
epochs=40
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
  callbacks=[WandbCallback()]
)

Epoch 1/40


InvalidArgumentError: ignored

In [ ]:
wandb.finish()

In [ ]:
!nvidia-smi --query-gpu=gpu_name --format=csv

name
Tesla T4


In [ ]:
import os, sys
from PIL import Image